<a href="https://colab.research.google.com/github/u6yuvi/NER-System/blob/main/notebooks/Cross_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
!git clone https://github.com/ntropy-network/ML-tools.git

Cloning into 'ML-tools'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 131 (delta 46), reused 110 (delta 35), pack-reused 0
Receiving objects: 100% (131/131), 779.41 KiB | 5.13 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [141]:
!pip3 install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 20.0 MB/s eta 0:00:00


In [79]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [80]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [81]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
!ls

ML-tools		poetry.lock	README.md   tests
parallel_cross_encoder	pyproject.toml	Test.ipynb  Train.ipynb


In [83]:
%cd ML-tools/parallel_cross_encoder/

/content/ML-tools/parallel_cross_encoder/ML-tools/parallel_cross_encoder


In [84]:
!ls

parallel_cross_encoder	pyproject.toml	Test.ipynb  Train.ipynb
poetry.lock		README.md	tests


In [85]:
from parallel_cross_encoder.configs import TrainConfigs
from copy import deepcopy

# Training Configs

In [86]:
from typing import List

from psutil import cpu_count
from pydantic import BaseModel, ValidationError, validator


class TrainConfigs(BaseModel):
    no_cuda: bool = False
    parallelize_cross_encoder: bool = True
    validation_data_ratio: float = 0.3
    output_dir: str = "outputs"
    batch_size: int = 32
    weight_decay: float = 0.0001
    dataloader_num_workers: int = 2
    model_type: str = "bert"
    negative_labels_per_sample: int = 3  # For parralelized cross encoder only
    early_stopping_epochs: int = 6
    warmup_epochs: float = 1
    encoder_hidden_layers: int = 2
    decoder_hidden_layers: int = 1  # For bart model only
    epochs: int = 10
    validation_threshold: float = 0.5
    learning_rate: float = 5e-5
    weight_decay: float = 1e-2
    logging_steps: int = 50
    max_length: int = 256

    @validator("model_type")
    def validate_model_type(cls, v):
        if v not in ["bert", "bart"]:
            raise ValidationError('Value must be one of ["bert", "bart"]')
        return v

    @validator("dataloader_num_workers")
    def validate_dataloader_num_workers(cls, v):
        if v is None or v == -1:
            return cpu_count()
        return v


class TestConfigs(BaseModel):
    device: str = "cpu"
    model_path: str
    batch_size: int = 16
    num_labels_per_sample: int = 4  # For parallelized cross-encoders only
    dataloader_num_workers: int = 2

    @validator("dataloader_num_workers")
    def validate_dataloader_num_workers(cls, v):
        if v is None or v == -1:
            return cpu_count()
        return v

In [143]:
configs = TrainConfigs(no_cuda=True, parallelize_cross_encoder=False, model_type='bert', output_dir="outputs", encoder_hidden_layers=2, epochs=6, batch_size=32, dataloader_num_workers=2, learning_rate=1e-04, early_stopping_epochs=6, validation_data_ratio=0.15, max_length=128)
configs

TrainConfigs(no_cuda=True, parallelize_cross_encoder=False, validation_data_ratio=0.15, output_dir='outputs', batch_size=32, weight_decay=0.01, dataloader_num_workers=2, model_type='bert', negative_labels_per_sample=3, early_stopping_epochs=6, warmup_epochs=1, encoder_hidden_layers=2, decoder_hidden_layers=1, epochs=6, validation_threshold=0.5, learning_rate=0.0001, logging_steps=50, max_length=128)

In [88]:
if configs.parallelize_cross_encoder:
    name = "parallelized_cross_encoder"
else:
  name = "vanilla_cross_encoder"
name = name + "_" + configs.model_type

name

'vanilla_cross_encoder_bert'

In [89]:
configs

TrainConfigs(no_cuda=False, parallelize_cross_encoder=False, validation_data_ratio=0.15, output_dir='outputs', batch_size=32, weight_decay=0.01, dataloader_num_workers=2, model_type='bert', negative_labels_per_sample=3, early_stopping_epochs=6, warmup_epochs=1, encoder_hidden_layers=2, decoder_hidden_layers=1, epochs=6, validation_threshold=0.5, learning_rate=0.0001, logging_steps=50, max_length=128)

In [90]:
from transformers.models.distilbert.modeling_distilbert import DistilBertForSequenceClassification
from transformers import AutoModel, AutoTokenizer

In [38]:
#DistilBert pooling Layer
import torch
#bs,seq_len,dim
hidden_state = torch.tensor([[[2]*768]*512]*32)
print(hidden_state.shape)
hidden_state[:0] #taking the output representation from CLS token


torch.Size([32, 512, 768])


torch.Size([0, 512, 768])

In [145]:
# Vanilla Distilbert

model_type = configs.model_type
encoder_hidden_layers = configs.encoder_hidden_layers
decoder_hidden_layers = configs.decoder_hidden_layers
multilabel = configs.parallelize_cross_encoder
print(multilabel,model_type)
if model_type == "bert":
    model_name = "typeform/distilbert-base-uncased-mnli"
    if multilabel:
        #model_class = DistilBertForMultiSequenceClassification
        pass
    else:
        model_class = DistilBertForSequenceClassification
    aux = AutoModel.from_pretrained(model_name)
    config = deepcopy(aux.config)
    config._num_labels = 2
    config.label2id = {"contradiction": 0, "entailment": 1}
    config.id2label = {0: "contradiction", 1: "entailment"}
    config.classif_dropout = 0.2
    config.classifier_dropout = 0.2
    if encoder_hidden_layers<1:
      raise ValueError(f'Encoder Hidden Layers {encoder_hidden_layers} not supported. It should be greater than 0')
    config.num_hidden_layers = encoder_hidden_layers #config part of PreTrained Config https://github.com/huggingface/transformers/blob/main/src/transformers/configuration_utils.py#L48:7
    model = model_class(config)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model.cls_token_id = tokenizer.cls_token_id
model.pad_token_id = tokenizer.pad_token_id
model.sep_token_id = tokenizer.sep_token_id

aux_state_dict = aux.state_dict()

# from collections import OrderedDict

# tag = "distilbert."
# new_dict = OrderedDict()
# for k, v in model.state_dict().items():
#     if k[len(tag) :] in aux_state_dict:
#         if "classif" not in k:
#             new_dict[k] = deepcopy(aux_state_dict[k[len(tag) :]])
#         else:
#             new_dict[k] = v
#     else:
#         new_dict[k] = v
# model.load_state_dict(new_dict)



The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


False bert


Some weights of the model checkpoint at typeform/distilbert-base-uncased-mnli were not used when initializing DistilBertModel: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument h

In [130]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [106]:
#model.state_dict().keys(), #aux.state_dict().keys()

In [107]:
# for k, v in aux_state_dict.items():
#   print(k)

# Dataset

In [109]:
!pip3 install datasets
from datasets import load_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.7 MB/s eta 0:00:00


In [110]:
newsgroups_data = load_dataset("ag_news")["train"]
newsgroups_data = newsgroups_data.shuffle(seed=42)


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [111]:
split = int(len(newsgroups_data) * (1 - configs.validation_data_ratio))
train_data, val_data = newsgroups_data[:split], newsgroups_data[split:]

In [116]:
train_data.keys()

dict_keys(['text', 'label'])

In [117]:
train_data["text"][0]

'Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.'

In [129]:
train_data["label"][0]

0

In [113]:
LABELS = [
    "world",
    "sports",
    "business",
    "science and technology",
]

HYPOTHESIS = "This example is about"

In [127]:
2//len(train_data["label"])

0

In [126]:
len(train_data["label"])

102000

In [112]:
class NewsDatasetSingleLabel:
    """
    This is the basic dataset for the vanilla cross-encoder (1 label per sample)
    """

    def __init__(
        self,
        samples,
        targets,
        target_names,
        tokenizer,
        model_type,
        max_length,
        entailment_id=1,
        contradiction_id=0,
    ):
        self.samples = samples
        self.targets = targets
        self.target_names = target_names

        self.model_type = model_type
        self.max_length = max_length
        self.entailment_id = entailment_id
        self.contradiction_id = contradiction_id
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.samples) * len(self.target_names)

    def __getitem__(self, idx):
        sample_idx = idx // len(self.target_names)
        label_idx = idx % len(self.target_names)
        prompt = self.samples[sample_idx]
        target_idx = self.targets[sample_idx]

        label = self.target_names[label_idx]
        if target_idx == label_idx:
            # pos label
            target = [self.entailment_id]
        else:
            # neg label
            target = [self.contradiction_id]

        hypothesis = f"{HYPOTHESIS} {label}"
        enc = self.tokenizer(
            prompt,
            hypothesis,
            truncation="only_first",
            max_length=self.max_length,
            return_tensors="pt",
        )
        enc["labels"] = torch.tensor(target, dtype=torch.long)
        enc["sample_idxs"] = torch.tensor(sample_idx, dtype=torch.long)

        return {k: v.squeeze() for k, v in enc.items()}

In [121]:
ds_train = NewsDatasetSingleLabel(
    samples=train_data["text"],
    targets=train_data["label"],
    target_names=LABELS,
    tokenizer=tokenizer,
    model_type=configs.model_type,
    max_length=configs.max_length,
)

In [124]:
next(iter(ds_train))

{'input_ids': tensor([  101,  7269, 11498,  2135,  6924,  2011,  9326,  4559, 10134,  2031,
          2716,  2116,  4865,  1998,  3655,  1999,  7269,  2000,  1037,  9190,
          1010,  1996,  2154,  2044,  2324,  2111,  2351,  1999, 18217,  2012,
          1037,  2576,  8320,  1012,   102,  2023,  2742,  2003,  2055,  2088,
           102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor(1),
 'sample_idxs': tensor(0)}

In [131]:
ds_val = NewsDatasetSingleLabel(
    samples=val_data["text"],
    targets=val_data["label"],
    target_names=LABELS,
    tokenizer=tokenizer,
    model_type=configs.model_type,
    max_length=configs.max_length,
)

In [132]:
from torch.utils.data import Dataset, DataLoader
from transformers.data.data_collator import DataCollatorWithPadding
train_dataloader = DataLoader(
    ds_train,
    batch_size=configs.batch_size,
    collate_fn=DataCollatorWithPadding(tokenizer, pad_to_multiple_of=2),
    shuffle=True,
    drop_last=True,
    num_workers=configs.dataloader_num_workers)

In [137]:
for idx,j in enumerate(train_dataloader):
  print(j)
  break

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  101,  3066,  2006,  ...,     0,     0,     0],
        [  101, 22851,  3022,  ...,     0,     0,     0],
        [  101,  8478, 28363,  ...,     0,     0,     0],
        ...,
        [  101,  3648,  7929,  ...,     0,     0,     0],
        [  101,  2064, 12722,  ...,     0,     0,     0],
        [  101,  2634,  1005,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0]), 'sample_idxs': tensor([61988, 97098, 83064,  2289, 82626, 65755, 66715, 58067, 30620,  3108,
        42307, 67598, 80934, 34280,  4789, 24635, 45913, 43540, 41766, 96998,
        87083, 13381, 31621,  2164, 97721, 13310, 72975, 64178, 43384, 37254,
        28899

In [146]:
for idx,batch in enumerate(train_dataloader):
  input_ids = batch["input_ids"]
  attention_mask = batch["attention_mask"]
  labels = batch["labels"]
  outputs = model(input_ids, attention_mask, labels)
    # take only the entailment output in case of non-parallel model:
  if not configs.parallelize_cross_encoder:
      outputs.logits = outputs.logits[:, 1]
  break
outputs


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: ignored

In [ ]:
def run_example(self, input_ids, attention_mask, labels=None):
    if len(input_ids.shape) == 3:
        input_ids = input_ids.squeeze(0)
        attention_mask = attention_mask.squeeze(0)
        if labels is not None:
            labels = labels.squeeze(0)
    outputs = self.model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels,
    )
    # take only the entailment output in case of non-parallel model:
    if not self.configs.parallelize_cross_encoder:
        outputs.logits = outputs.logits[:, 1]
    return outputs

def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", outputs.loss.item(), prog_bar=True, logger=True)
    return outputs.

In [144]:
from parallel_cross_encoder.train import train_cmd
train_cmd(configs)



 ******************************** 
*	Beginning Training	*
 ********************************
TRAIN CONFIGS : no_cuda=True parallelize_cross_encoder=False validation_data_ratio=0.15 output_dir='outputs' batch_size=32 weight_decay=0.01 dataloader_num_workers=2 model_type='bert' negative_labels_per_sample=3 early_stopping_epochs=6 warmup_epochs=1 encoder_hidden_layers=2 decoder_hidden_layers=1 epochs=6 validation_threshold=0.5 learning_rate=0.0001 logging_steps=50 max_length=128


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


MODEL TRAINED CONFIGS : no_cuda=True parallelize_cross_encoder=False validation_data_ratio=0.15 output_dir='outputs' batch_size=32 weight_decay=0.01 dataloader_num_workers=2 model_type='bert' negative_labels_per_sample=3 early_stopping_epochs=6 warmup_epochs=1 encoder_hidden_layers=2 decoder_hidden_layers=1 epochs=6 validation_threshold=0.5 learning_rate=0.0001 logging_steps=50 max_length=128


Some weights of the model checkpoint at typeform/distilbert-base-uncased-mnli were not used when initializing DistilBertModel: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument h

MODEL : LightningModel(
  (model): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
           

...

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Validation :: acc: 0.75, pre: 0.0, rec: 0.0, f1: 0.0


Training: 0it [00:00, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


''